# SERLI - AI4Industry - 2025

---
Notebook du Groupe rouge foncé (on aurait préféré le bleu)
---

## Import des librairies

In [45]:
import cv2
import os
import json
from tqdm import tqdm
import numpy as np
import threading
from scipy.ndimage import label

## Variables globales

In [39]:
DATASET_FOLDER = "dataset/"
IMAGE_SIZE = (848, 480)
MASK_FILE = "mask1.png"
NB_MASKED_IMAGES = 24

## Variables système

## Import des données

In [ ]:
data_lock = threading.Lock() 
image_list = []
label_list = []

mask = cv2.imread(MASK_FILE, 0)

def read_subfolder(subfolder_path, image_size, image_list, label_list):
    print(f"Reading subfolder: {subfolder_path}")
    for file in os.listdir(subfolder_path):
        if file.endswith('.png'):
            image_path = os.path.join(subfolder_path, file)
            file_name = file.split(".")[0]
            json_path = os.path.join(subfolder_path, file_name + ".json")

            index = int(file_name.split("_")[1])

            if os.path.exists(json_path) and os.path.exists(image_path):
                image = cv2.imread(image_path)
                image = cv2.resize(image, image_size)
                image = cv2.bitwise_and(image, image, mask=mask)

                with data_lock:
                    image_list[index] = image
                    with open(json_path) as json_file:
                        data = json.load(json_file)
                        label_list[index] = data
            else:
                print(f"json file does not exist for: {file}")
    print(f"Finished reading subfolder: {subfolder_path}")

if os.path.exists(DATASET_FOLDER):
    print("Dataset folder exists")
    
    dataset_len = 1
    for subfolders in os.listdir(DATASET_FOLDER):
        subfolder_path = os.path.join(DATASET_FOLDER, subfolders)
        for file in os.listdir(subfolder_path):
            if file.endswith('.png'):
                dataset_len += 1
                
    image_list = np.zeros((dataset_len, IMAGE_SIZE[1], IMAGE_SIZE[0], 3), dtype=np.uint8)
    label_list = np.zeros((dataset_len, 1), dtype=object)
    
    threads = []

    # Lancer un thread pour chaque sous-dossier
    for subfolder in os.listdir(DATASET_FOLDER):
        subfolder_path = os.path.join(DATASET_FOLDER, subfolder)
        if os.path.isdir(subfolder_path):
            thread = threading.Thread(target=read_subfolder, args=(subfolder_path, IMAGE_SIZE, image_list, label_list))
            threads.append(thread)
            thread.start()

    # Attendre que tous les threads soient terminés
    for thread in threads:
        thread.join()
else:
    raise Exception("Dataset folder does not exist")


In [79]:
colors_lst = [ 
    [255, 0, 0], [0, 255, 0], [0, 0, 255], [255, 255, 0], 
    [255, 0, 255], [0, 255, 255], [128, 0, 0], [0, 128, 0], 
    [0, 0, 128], [128, 128, 0], [128, 0, 128], [0, 128, 128], 
    [192, 0, 0], [0, 192, 0], [0, 0, 192], [192, 192, 0], 
    [192, 0, 192], [0, 192, 192], [64, 0, 0], [0, 64, 0], 
    [0, 0, 64], [64, 64, 0], [64, 0, 64], [0, 64, 64]
]

def segment_on_dt(a, img):
    border = cv2.dilate(img, None, iterations=5)
    border = border - cv2.erode(border, None)

    dt = cv2.distanceTransform(img, 2, 3)
    dt = ((dt - dt.min()) / (dt.max() - dt.min()) * 255).astype(np.uint8)
    _, dt = cv2.threshold(dt, 180, 255, cv2.THRESH_BINARY)
    lbl, ncc = label(dt)
    lbl = lbl * (255 / (ncc + 1))
    # Completing the markers now. 
    lbl[border == 255] = 255

    lbl = lbl.astype(np.int32)
    cv2.watershed(a, lbl)

    lbl[lbl == -1] = 0
    lbl = lbl.astype(np.uint8)
    return 255 - lbl

def apply_watershed(image):
    
    img_blur = cv2.GaussianBlur(image, (3, 3), 0)
    
    img_gray = cv2.cvtColor(img_blur, cv2.COLOR_BGR2GRAY)    
    _, img_bin = cv2.threshold(img_gray, 0, 255,
            cv2.THRESH_OTSU)
    img_bin = cv2.morphologyEx(img_bin, cv2.MORPH_OPEN,
            np.ones((3, 3), dtype=int), iterations=4)

    result = segment_on_dt(image, img_bin)
    
    return result

In [80]:
def apply_kmeans(image, k = 5) :
    Z = image.reshape((-1,3))
    Z = np.float32(Z)
    
    criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 10, 1.0)
    K = k
    ret,label,center=cv2.kmeans(Z, k, None, criteria, 10, cv2.KMEANS_RANDOM_CENTERS)
    
    center = np.uint8(center)
    res = center[label.flatten()]
    res2 = res.reshape((image.shape))
    
    return res2

In [81]:
def apply_grabcut(image):
    mask = np.zeros(image.shape[:2], np.uint8)
    bgdModel = np.zeros((1, 65), np.float64)
    fgdModel = np.zeros((1, 65), np.float64)
    
    cv2.grabCut(image, mask, None, bgdModel, fgdModel, 5, cv2.GC_INIT_WITH_RECT)
    
    mask2 = np.where((mask == 2) | (mask == 0), 0, 1).astype('uint8')
    
    image[mask2 == 0] = [0, 0, 0]
    
    return image

In [ ]:
# Display the segmented images as a video stream
for img in image_list:
    
    cv2.imshow('Segmented Image', apply_watershed(img))
    if cv2.waitKey(18) & 0xFF == ord('q'):
        break
    elif cv2.waitKey(18) & 0xFF == ord('s'):
        cv2.imwrite("segmented_image.png", apply_watershed(img))
    

cv2.destroyAllWindows()